**TODO: change file path to folders with the output data from the different models**


Note: all out output files should be text files

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir('/content/drive/Shareddrives/CS260-Project')

In [ ]:
import numpy as np
import re
from nltk.translate.gleu_score import sentence_gleu
import copy
import math
from collections import ChainMap
from numpy.lib.twodim_base import vander


In [ ]:
!cd ./drive/Shareddrives
!ls ./data/out/gpt3

/bin/bash: line 0: cd: ./drive/Shareddrives: No such file or directory
'Example Outputs.gdoc'	 few-shot   fine-tunning


In [ ]:
def get_gleu_score(sentence, remaining_sentences):
    lst = []
    for i in remaining_sentences:
        gleu = sentence_gleu(sentence, i)
        lst.append(gleu)
    return lst

def calculate_selfGleu(sentences):
    gleu_scores = []
    for i in sentences:
        sentences_copy = copy.deepcopy(sentences)
        remaining_sentences = sentences_copy.remove(i)
        gleu = get_gleu_score(i,sentences_copy)
        gleu_scores.append(gleu)
    return np.mean(gleu_scores)


def normalized_term_inverse_document_frequency(texts, k=0.5):
  normalized_term_frequency = []
  for text in texts:
    word_count = {}
    for word in text.split(' '):
      if word == '':
        pass
      elif word not in word_count:
        word_count[word] = 1
      elif word in word_count:
        word_count[word] += 1
    if len(word_count.values()) != 0:
      max_val = max(word_count.values())
      word_count = {key:(k + (1-k)*v/max_val) for key, v in word_count.items()}
      normalized_term_frequency.append(word_count)

  occurance = {}
  flattened_word_count = {key: v for d in normalized_term_frequency for key, v in d.items()}
  flattened_word_count = list(set(flattened_word_count.keys()))
  for word in flattened_word_count:
    for doc in normalized_term_frequency:
      if word in doc.keys():
        if word not in occurance:
          occurance[word] = 1
        elif word in occurance:
          occurance[word] += 1
  
  N = len(texts)
  idf = {key: np.log((N+1)/(v+1))+1 for key, v in occurance.items()}

  ntf_idf = []
  for word_frequency in normalized_term_frequency:
    term_frequency = {}
    for word in word_frequency:
      term_frequency[word] = word_frequency[word]*idf[word]
    ntf_idf.append(term_frequency)
  return ntf_idf


def type_token_ratio(texts):
  token_frequency = []
  for text in texts:
    word_count = {}
    for word in text.split(' '):
      if word == '':
        pass
      elif word not in word_count:
        word_count[word] = 1
      elif word in word_count:
        word_count[word] += 1
    token_frequency.append(word_count)

  type_token_ratio = []
  for word_count in token_frequency:
    if sum(word_count.values()) > 0:
      type_token_ratio.append(len(word_count)/sum(word_count.values()))
  return type_token_ratio


# GPT3 Evaluation

## GTP3 Few Shot Evaluation

In [ ]:
output_files = os.listdir("./data/out/gpt3/few-shot")
full_text = ""
for file in output_files:
  output = open("./data/out/gpt3/few-shot/" + file, "r")
  full_text += output.read() + "\n"

full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
full_text = re.sub(r"'", "", full_text)
texts = re.findall("<START>(.*?)<END>", full_text)
texts = list(map(str.lower, texts))
print(texts[0])



 bee gees - another lonely night in new york  another lonely night in new york the city streets all seem to talk of love that used to be a love that used to be  tonight the city feels so right each candle burning ever so bright the ghosts of love and memories the ghosts of love and memories  oh how i wish i could turn back time to see the love thats on your mind life would be so much easier life would be so much easier  again im here in the place of dreams but what i thought was here is gone its too late to change my mind its too late to change my mind  oh cant this feeling pass oh wont it last not all love needs a beginning and an end oh why cant we try again another lonely night in new york the city streets all seem to talk of love that used to be a love that used to be  tonight the city feels so right each candle burning ever so bright the ghosts of love and memories the ghosts of love and memories


In [ ]:
gleu = calculate_selfGleu(texts)
nfts = normalized_term_inverse_document_frequency(texts)
avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
ttr = type_token_ratio(texts)
avg_ttr = np.mean(ttr)

In [ ]:

evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

with open('./data/out/gpt3_few_shot_eval.txt', 'w') as f:
  f.write(evaluation_results)

print("Self-Gleu")
print(gleu)
print("Normalized Term Frequency Inverse Document Frequency")
print(nfts)
print("Average Normalized Term Frequency Inverse Document Frequency")
print(avg_nft)
print("Type Token Ratio")
print(ttr)
print("Average Type Token Ratio")
print(avg_ttr)


Self-Gleu
0.0002898936810041768
Normalized Term Frequency Inverse Document Frequency
[{'bee': 1.9391507543695408, 'gees': 1.9391507543695408, '-': 1.4351908518387555, 'another': 2.0281712985300593, 'lonely': 2.2917236188003662, 'night': 1.1270799638021305, 'in': 0.9299528468804251, 'new': 1.5776256311660948, 'york': 2.0281712985300593, 'the': 1.0408219945202553, 'city': 2.1841844753400634, 'streets': 2.1154371865849533, 'all': 1.0268820219144124, 'seem': 1.5656627474604603, 'to': 0.95, 'talk': 1.2129907485191886, 'of': 1.0832713188464962, 'love': 1.4462871026284194, 'that': 1.1577485271846648, 'used': 2.4680100510157787, 'be': 1.0628032535776288, 'a': 0.7330916914814253, 'tonight': 1.8721581217200545, 'feels': 1.8721581217200545, 'so': 1.2086604990127927, 'right': 1.5656627474604603, 'each': 1.8721581217200545, 'candle': 2.1154371865849533, 'burning': 1.699548878248986, 'ever': 1.2836605699130188, 'bright': 1.5656627474604603, 'ghosts': 2.4680100510157787, 'and': 0.75, 'memories': 2.18

## GPT3 Few Shot Evaluation Each Txt

In [ ]:
output_files = os.listdir("./data/out/gpt3/few-shot")
for file in output_files:
  output = open("./data/out/gpt3/few-shot/" + file , "r")
  full_text = output.read()

  full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
  full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
  full_text = re.sub(r"'", "", full_text)
  texts = re.findall("<START>(.*?)<END>", full_text)
  texts = list(map(str.lower, texts))
  print(texts[0])

  gleu = calculate_selfGleu(texts)
  nfts = normalized_term_inverse_document_frequency(texts)
  avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
  ttr = type_token_ratio(texts)
  avg_ttr = np.mean(ttr)

  evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

  with open('./data/out/gpt3_'+ file[4:-4]+'_eval.txt', 'w') as f:
    f.write(evaluation_results)

  print("Self-Gleu")
  print(gleu)
  print("Normalized Term Frequency Inverse Document Frequency")
  print(nfts)
  print("Average Normalized Term Frequency Inverse Document Frequency")
  print(avg_nft)
  print("Type Token Ratio")
  print(ttr)
  print("Average Type Token Ratio")
  print(avg_ttr)





 bee gees - another lonely night in new york  another lonely night in new york the city streets all seem to talk of love that used to be a love that used to be  tonight the city feels so right each candle burning ever so bright the ghosts of love and memories the ghosts of love and memories  oh how i wish i could turn back time to see the love thats on your mind life would be so much easier life would be so much easier  again im here in the place of dreams but what i thought was here is gone its too late to change my mind its too late to change my mind  oh cant this feeling pass oh wont it last not all love needs a beginning and an end oh why cant we try again another lonely night in new york the city streets all seem to talk of love that used to be a love that used to be  tonight the city feels so right each candle burning ever so bright the ghosts of love and memories the ghosts of love and memories
Self-Gleu
0.0002853783448498561
Normalized Term Frequency Inverse Document Frequency


## GPT3 Fine-Tunning Evaluation

In [ ]:
output_files = os.listdir("./data/out/gpt3/fine-tunning")
full_text = ""
for file in output_files:
  output = open("./data/out/gpt3/fine-tunning/" + file, "r")
  full_text += output.read() + "\n"

full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
full_text = re.sub(r"'", "", full_text)
texts = re.findall("<START>(.*?)<END>", full_text)
texts = list(map(str.lower, texts))
print(texts[0])



 rays to rust we both will balance our acts xxx all the youths chiding dancing on the flames they were burning xxx all the darkness of shining believe it feel the coming of the wings


In [ ]:
gleu = calculate_selfGleu(texts)
nfts = normalized_term_inverse_document_frequency(texts)
avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
ttr = type_token_ratio(texts)
avg_ttr = np.mean(ttr)

In [ ]:

evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

with open('./data/out/gpt3_fine_tunning_eval.txt', 'w') as f:
  f.write(evaluation_results)

print("Self-Gleu")
print(gleu)
print("Normalized Term Frequency Inverse Document Frequency")
print(nfts)
print("Average Normalized Term Frequency Inverse Document Frequency")
print(avg_nft)
print("Type Token Ratio")
print(ttr)
print("Average Type Token Ratio")
print(avg_ttr)


Self-Gleu
0.0003238284188636679
Normalized Term Frequency Inverse Document Frequency
[{'rays': 3.428414754540104, 'to': 0.6593424444353484, 'rust': 3.428414754540104, 'we': 1.1511212410367873, 'both': 2.405565899197049, 'will': 1.387696325542811, 'balance': 3.185135689675205, 'our': 1.7284867481063744, 'acts': 3.428414754540104, 'xxx': 3.9998172136301213, 'all': 1.0670588942116235, 'the': 1.0553136379840522, 'youths': 3.428414754540104, 'chiding': 3.428414754540104, 'dancing': 3.0125264462041366, 'on': 0.9237823926027217, 'flames': 2.878640315415611, 'they': 1.3584222270411517, 'were': 1.4970892596191834, 'burning': 2.405565899197049, 'darkness': 2.7692473813392384, 'of': 0.9637136950473781, 'shining': 2.878640315415611, 'believe': 1.9630065333185813, 'it': 0.9191846689553803, 'feel': 1.387696325542811, 'coming': 2.353359073003271, 'wings': 2.5966381378681693}, {'all': 0.8468721382631933, 'the': 1.0553136379840522, 'shes': 1.9363921437258307, 'ive': 1.2720528768503379, 'been': 1.292611

## GPT3 Fine-Tunning Evaluation Each Txt

In [ ]:
output_files = os.listdir("./data/out/gpt3/fine-tunning")
for file in output_files:
  output = open("./data/out/gpt3/fine-tunning/" + file , "r")
  full_text = output.read()

  full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
  full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
  full_text = re.sub(r"'", "", full_text)
  texts = re.findall("<START>(.*?)<END>", full_text)
  texts = list(map(str.lower, texts))
  print(texts[0])

  gleu = calculate_selfGleu(texts)
  nfts = normalized_term_inverse_document_frequency(texts)
  avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
  ttr = type_token_ratio(texts)
  avg_ttr = np.mean(ttr)

  evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

  with open('./data/out/gpt3_'+ file[4:-4]+'_eval.txt', 'w') as f:
    f.write(evaluation_results)

  print("Self-Gleu")
  print(gleu)
  print("Normalized Term Frequency Inverse Document Frequency")
  print(nfts)
  print("Average Normalized Term Frequency Inverse Document Frequency")
  print(avg_nft)
  print("Type Token Ratio")
  print(ttr)
  print("Average Type Token Ratio")
  print(avg_ttr)





 rays to rust we both will balance our acts xxx all the youths chiding dancing on the flames they were burning xxx all the darkness of shining believe it feel the coming of the wings
Self-Gleu
0.00030265518852253854
Normalized Term Frequency Inverse Document Frequency
[{'rays': 1.675055681536833, 'to': 0.6, 'rust': 1.675055681536833, 'we': 0.9233979004396121, 'both': 1.4317766166719343, 'will': 1.12528124241234, 'balance': 1.675055681536833, 'our': 1.4317766166719343, 'acts': 1.675055681536833, 'xxx': 1.9542316284596384, 'all': 0.7609079638927407, 'the': 1.0, 'youths': 1.675055681536833, 'chiding': 1.675055681536833, 'dancing': 1.675055681536833, 'on': 0.7093929340763728, 'flames': 1.4317766166719343, 'they': 1.015888308335967, 'were': 1.015888308335967, 'burning': 1.4317766166719343, 'darkness': 1.675055681536833, 'of': 0.8276250897557682, 'shining': 1.675055681536833, 'believe': 1.259167373200866, 'it': 0.7093929340763728, 'feel': 0.7726092434710684, 'coming': 1.675055681536833, 'win

## GPT-3 Fine-Tuning Evaluation (larger datasets/more topics)

In [ ]:
output_files = os.listdir("./data/out/gpt3/fine-tunning/big-data-set")
for file in output_files:
  output = open("./data/out/gpt3/fine-tunning/big-data-set/" + file , "r")
  full_text = output.read()

  full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
  full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
  full_text = re.sub(r"'", "", full_text)
  texts = re.findall("<START>(.*?)<END>", full_text)
  texts = list(map(str.lower, texts))
  print(texts[0])

  gleu = calculate_selfGleu(texts)
  nfts = normalized_term_inverse_document_frequency(texts)
  avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
  ttr = type_token_ratio(texts)
  avg_ttr = np.mean(ttr)

  evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

  with open('./data/out/gpt3_'+ file[4:-4]+'_eval.txt', 'w') as f:
    f.write(evaluation_results)

  print("Self-Gleu")
  print(gleu)
  print("Normalized Term Frequency Inverse Document Frequency")
  print(nfts)
  print("Average Normalized Term Frequency Inverse Document Frequency")
  print(avg_nft)
  print("Type Token Ratio")
  print(ttr)
  print("Average Type Token Ratio")
  print(avg_ttr)


 im standing there in the final cold long before our old den of arctic lies waste with black footprint hills and non-majestic clouds frozen bloody sea  and i hear a terrible laughter its the last echo of thy woe and so the end ill gaze on the smoke and shrouds of undone grief  i would to the earth would lay my soul to the darkness by the side of my love till next the breath revives and i found the realm of neveetherkelti and thir broneless souls i walk in silence cold iron evermore  letho i pass by thy forests as restless shades of snow in lonely raths abandoned my sorrows like sighs and passing woe and i sing a low song of pity in the echeos of distant abyss  dared by my virgin to the dark abyss i grieve over the endless tomb i visit unknown father and many unnamable in forms of shadowy nothing  im doomed to wander lost wandering in shadow iim to the shades of evermore alone i wander so through deepest realms of mourning deeply haunting deep lost souls hath infected thee dwell deep in

# GPT2 Evaluation

## GPT2 100 Total Evaluation

In [ ]:
output_files = os.listdir("./data/out/gpt2_100")
full_text = ""
for file in output_files:
  output = open("./data/out/gpt2_100/" + file, "r")
  full_text += output.read() + "\n"

full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
full_text = re.sub(r"'", "", full_text)
texts = re.findall("<START>(.*?)<END>", full_text)
texts = list(map(str.lower, texts))
print(texts[0])



 0 0 0 0 0 2 9 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 


In [ ]:
gleu = calculate_selfGleu(texts)
nfts = normalized_term_inverse_document_frequency(texts)
avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
ttr = type_token_ratio(texts)
avg_ttr = np.mean(ttr)

In [ ]:

evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

with open('./data/out/gpt2_100_eval.txt', 'w') as f:
  f.write(evaluation_results)

print("Self-Gleu")
print(gleu)
print("Normalized Term Frequency Inverse Document Frequency")
print(nfts)
print("Average Normalized Term Frequency Inverse Document Frequency")
print(avg_nft)
print("Type Token Ratio")
print(ttr)
print("Average Type Token Ratio")
print(avg_ttr)


Self-Gleu
0.010357739514897032
Normalized Term Frequency Inverse Document Frequency
[{'0': 2.158463350316115, '2': 1.6631688273899552, '9': 1.5716282124163237, '3': 3.0501711593797225}, {'0': 4.228826155721369, '0007': 2.4806745614857824, '0044': 2.4806745614857824, '-0': 2.4806745614857824, '10%': 2.4806745614857824, '00%': 4.803845976210563, '00': 2.4806745614857824, 'u2': 2.4806745614857824, '1': 1.926973967997055, '0"': 2.4806745614857824, 'width="0"': 2.4806745614857824, 'height="0"><a': 2.4806745614857824, 'href="': 2.5397382415211585, '"': 1.7636343047846008, 'target="_blank">[url=http://www': 2.5397382415211585, 'mediaite': 2.6184898149016593, 'com/file/e8f9e5fc-5bf0-4578-9e2f-4c8a-8f8f67b8d8c3d': 2.4806745614857824, 'jpg]http://www': 2.5397382415211585, 'com/file/9d2ce8f8-4df6-4f3b-8be4-0ccd-2b0df9cd8b0a0': 2.4806745614857824, 'jpg[/url]</a></p>': 2.520050348176033, '<p': 2.520050348176033, 'align="left"': 2.520050348176033, 'width="100%"': 2.520050348176033, 'height="100%"': 

## GPT2 100 Each Topic


In [ ]:
output_files = os.listdir("./data/out/gpt2_100_each_formatted")
full_text = ""
for file in output_files:
  output = open("./data/out/gpt2_100_each_formatted/" + file, "r")
  full_text += output.read() + "\n"

full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
full_text = re.sub(r"'", "", full_text)
texts = re.findall("<START>(.*?)<END>", full_text)
texts = list(map(str.lower, texts))
print(texts[0])



  ive lost everything i know now that my face is on fire i know it was made of stone my head was burnt out now i know what was inside its burning me out and now im feeling a strong desire for power    my voice is broken now my head is broken im breaking my neck  im crying out in pain now im screaming for mercy  my blood is boiling inside my own neck       i am the only one in the world to know how to make it out of nothing     i am the only one in the world who can feel my rage and fear i am the only one in the world who can feel my rage and fear  i am the only one in the world who can feel my rage and fear    i am the only one in the world who can feel the pain of my own face    i am the only one in the world who can feel my fear        the last man alive cant stand to be king          my name is oryx my name is the only one in the world who can understand how to make it out of nothing    i am the only one in the world who can feel my rage and fear i am the only one in the world who c

In [ ]:
gleu = calculate_selfGleu(texts)
nfts = normalized_term_inverse_document_frequency(texts)
avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
ttr = type_token_ratio(texts)
avg_ttr = np.mean(ttr)

In [ ]:

evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

with open('./data/out/gpt2_100_eval.txt', 'w') as f:
  f.write(evaluation_results)

print("Self-Gleu")
print(gleu)
print("Normalized Term Frequency Inverse Document Frequency")
print(nfts)
print("Average Normalized Term Frequency Inverse Document Frequency")
print(avg_nft)
print("Type Token Ratio")
print(ttr)
print("Average Type Token Ratio")
print(avg_ttr)


Self-Gleu
0.0027109212024989332
Normalized Term Frequency Inverse Document Frequency
[{'ive': 1.5590223372823584, 'lost': 1.915498030141759, 'everything': 2.0634488102598176, 'i': 1.1333116562411254, 'know': 1.2070137216466854, 'now': 1.896636797111575, 'that': 1.2357378838651374, 'my': 1.6085234278559601, 'face': 2.120766832767035, 'is': 1.334058304640013, 'on': 1.4442627966064792, 'fire': 2.0634488102598176, 'it': 1.3820733036435346, 'was': 1.5244996186401725, 'made': 2.0634488102598176, 'of': 1.3387160408538992, 'stone': 2.2719737230011594, 'head': 2.120766832767035, 'burnt': 2.2719737230011594, 'out': 1.7322470414248428, 'what': 1.3093326034517696, 'inside': 2.3350841041956363, 'its': 1.4984482046590757, 'burning': 2.2719737230011594, 'me': 1.202546644422958, 'and': 1.1706881086683052, 'im': 1.058729900658188, 'feeling': 1.8007384894658798, 'a': 0.8623988535539134, 'strong': 2.2719737230011594, 'desire': 2.2719737230011594, 'for': 1.3880112418893784, 'power': 2.2719737230011594, 'v

## GPT2 100 Total Evaluation Per Topic

In [ ]:
output_files = os.listdir("./data/out/gpt2_100")
num_topics = len(output_files)
for topic in range(num_topics):
  output = open("./data/out/gpt2_100/val-output-100-topic-" + str(topic) + ".txt", "r")
  full_text = output.read()

  full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
  full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
  full_text = re.sub(r"'", "", full_text)
  texts = re.findall("<START>(.*?)<END>", full_text)
  texts = list(map(str.lower, texts))
  print(texts[0])

  gleu = calculate_selfGleu(texts)
  ntfidf = normalized_term_inverse_document_frequency(texts)
  avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
  ttr = type_token_ratio(texts)
  avg_ttr = np.mean(ttr)

  evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

  with open('./data/out/gpt2_100_'+ str(topic)+'_eval.txt', 'w') as f:
    f.write(evaluation_results)

  print("Self-Gleu")
  print(gleu)
  print("Normalized Term Frequency Inverse Document Frequency")
  print(ntfidf)
  print("Average Normalized Term Frequency Inverse Document Frequency")
  print(avg_nft)
  print("Type Token Ratio")
  print(ttr)
  print("Average Type Token Ratio")
  print(avg_ttr)





 0 0 0 0 0 2 9 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 
Self-Gleu
0.0002452735595434571
Normalized Term Frequency Inverse Document Frequency
[{'0': 0.8642105400774721, '2': 1.0536782532687803, '9': 1.0536782532687803, '3': 2.09861228866811}, {'0': 1.6931471805599454, '0007': 1.0577005934887274, '0044': 1.0577005934887274, '-0': 1.0577005934887274, '10%': 1.0577005934887274, '00%': 2.0482455937400754, '00': 1.0577005934887274, 'u2': 1.0577005934887274, '1': 1.0577005934887274, '0"': 1.0577005934887274, 'width="0"': 1.0577005934887274, 'height="0"><a': 1.0

## GPT2 100 Each Topic Evaluation per Topic


In [ ]:
output_files = os.listdir("./data/out/gpt2_100_each_formatted")
num_topics = len(output_files)
for topic in range(num_topics):
  output = open("./data/out/gpt2_100_each_formatted/val-output-100-each-topic-" + str(topic) + ".txt", "r")
  full_text = output.read()

  full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
  full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
  full_text = re.sub(r"'", "", full_text)
  texts = re.findall("<START>(.*?)<END>", full_text)
  texts = list(map(str.lower, texts))
  print(texts[0])

  gleu = calculate_selfGleu(texts)
  nfts = normalized_term_inverse_document_frequency(texts)
  avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
  ttr = type_token_ratio(texts)
  avg_ttr = np.mean(ttr)

  evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

  with open('./data/out/gpt2_100_each_'+ str(topic)+'_eval.txt', 'w') as f:
    f.write(evaluation_results)

  print("Self-Gleu")
  print(gleu)
  print("Normalized Term Frequency Inverse Document Frequency")
  print(nfts)
  print("Average Normalized Term Frequency Inverse Document Frequency")
  print(avg_nft)
  print("Type Token Ratio")
  print(ttr)
  print("Average Type Token Ratio")
  print(avg_ttr)





  ive lost everything i know now that my face is on fire i know it was made of stone my head was burnt out now i know what was inside its burning me out and now im feeling a strong desire for power    my voice is broken now my head is broken im breaking my neck  im crying out in pain now im screaming for mercy  my blood is boiling inside my own neck       i am the only one in the world to know how to make it out of nothing     i am the only one in the world who can feel my rage and fear i am the only one in the world who can feel my rage and fear  i am the only one in the world who can feel my rage and fear    i am the only one in the world who can feel the pain of my own face    i am the only one in the world who can feel my fear        the last man alive cant stand to be king          my name is oryx my name is the only one in the world who can understand how to make it out of nothing    i am the only one in the world who can feel my rage and fear i am the only one in the world who c

## GPT-2 20 Topics

In [ ]:
output_files = os.listdir("./data/out/gpt-2-20-topics")
for file in output_files:
  output = open("./data/out/gpt-2-20-topics/" + file , "r")
  full_text = output.read()

  full_text = full_text.replace('\n', ' ').replace('\r', '').replace('###', '').replace('nnn', '')
  full_text = re.sub(r"[,.;@#?!&$]+\ *", " ", full_text)
  full_text = re.sub(r"'", "", full_text)
  texts = re.findall("<START>(.*?)<END>", full_text)
  texts = list(map(str.lower, texts))
  print(texts[0])

  gleu = calculate_selfGleu(texts)
  nfts = normalized_term_inverse_document_frequency(texts)
  avg_nft = np.mean([np.mean(list(nft.values())) for nft in nfts])
  ttr = type_token_ratio(texts)
  avg_ttr = np.mean(ttr)

  evaluation_results = "Self-Gleu \n" + str(gleu) + "\n\nAverage Normalized Term Frequency Inverse Document Frequency \n" + str(avg_nft) + "\n\nAverage Type Token Ratio \n" + str(avg_ttr) + "\n\nNormalized Term Frequency Inverse Document Frequency \n" + str(nfts) + "\n\nType Token Ratio \n" + str(ttr)

  with open('./data/out/gpt2_'+ file[:-4]+'_eval.txt', 'w') as f:
    f.write(evaluation_results)

  print("Self-Gleu")
  print(gleu)
  print("Normalized Term Frequency Inverse Document Frequency")
  print(nfts)
  print("Average Normalized Term Frequency Inverse Document Frequency")
  print(avg_nft)
  print("Type Token Ratio")
  print(ttr)
  print("Average Type Token Ratio")
  print(avg_ttr)